In [1]:
%%capture
!pip install unsloth
!pip install transformers dataset

# Setting up LLaMA 3.2 model

In [ ]:
# Models Loading
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Unsloths auto supports RoPE Scaling internally!
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B", # or choose "unsloth/Llama-3.2-1B" # base version of Llama-3.2
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [ ]:
# PEFT settings
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining],
    lora_alpha = 32, # Suggested rx2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/usr/local/lib/python3.10/dist-packages/unsloth/models/_utils.py:888: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.10.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


# Continual Pretraining

In [ ]:
from datasets import load_dataset

alpaca_dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

# # tách lấy một phần
# alpaca_dataset = alpaca_dataset['train'].select(range(5000))

README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

(…)-00000-of-00001-5e7cb295b9cff0bf.parquet:   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [ ]:
# Format
alpaca_prompt = """Dưới đây là hướng dẫn mô tả một nhiệm vụ. Viết một phản hồi hoàn thành yêu cầu một cách thích hợp.

### Nhiệm vụ:
{}

### Đầu vào:
{}

### Câu trả lời:
{}"""

EOS_TOKEN = tokenizer.eos_token # end with eos
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

alpaca_dataset = alpaca_dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [ ]:
# Print checking
print(alpaca_dataset['train'][0]) # Select the 'train' split before indexing

{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!', 'output': 'Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea

# Continual Pretraining

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments


# training mode
FastLanguageModel.for_training(model)

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=alpaca_dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # Use num_train_epochs and warmup_ratio for longer runs!
        max_steps = 120,
        warmup_steps = 10,
        # warmup_ratio = 0.1,
        # num_train_epochs = 1,

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=8):   0%|          | 0/112165 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!
Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for embed_tokens.
Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for lm_head.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 112,165 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 120
 "-____-"     Number of trainable parameters = 812,318,720
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.979500
2,3.086200
3,2.951200
4,2.851400
5,2.867200
6,2.863600
7,2.941400
8,2.740500
9,2.804600
10,2.780100


test

In [ ]:
# Inference Testing
## Inference Mode
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    alpaca_prompt.format(
        "If you are a doctor, please answer the medical questions based on the patient's description.", # instruction
        "I woke up feel nauseous.", # input - leave blank if there is no input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Dưới đây là hướng dẫn mô tả một nhiệm vụ. Viết một phản hồi hoàn thành yêu cầu một cách thích hợp.

### Nhiệm vụ:
If you are a doctor, please answer the medical questions based on the patient's description.

### Đầu vào:
I woke up feel nauseous.

### Câu trả lời:
Hi, Thanks for your query. I can understand your concern. Nausea is a common symptom of many diseases. It can be due to many reasons like motion sickness, stress, anxiety, pregnancy, food poisoning, viral infection, etc. It can also be due to some underlying diseases like gastritis, peptic ulcer, etc. So, it is better to consult your doctor for proper diagnosis and treatment. Hope I have answered your query. Let me know if I can assist you further. Take care.<|end_of_text|>


In [ ]:
# Lưu mô hình đã fine-tune
model.save_pretrained('./fine_tuned_llama3b')
tokenizer.save_pretrained('./fine_tuned_llama3b')

('./fine_tuned_llama3b/tokenizer_config.json',
 './fine_tuned_llama3b/special_tokens_map.json',
 './fine_tuned_llama3b/tokenizer.json')

# connect drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Connect tele

In [3]:
%%capture
!pip install -U python-telegram-bot
!pip install nest_asyncio

In [ ]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from transformers import AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch
import nest_asyncio

# Khắc phục lỗi vòng lặp sự kiện
nest_asyncio.apply()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Đường dẫn mô hình và tokenizer đã fine-tune
model_name = './fine_tuned_llama3b'

# Tải mô hình và tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model, _ = FastLanguageModel.from_pretrained(model_name)
model = model.to(device) # Move only the model to the device

FastLanguageModel.for_inference(model)

# Định nghĩa prompt Alpaca
alpaca_prompt = """Dưới đây là hướng dẫn mô tả một nhiệm vụ. Viết một phản hồi hoàn thành yêu cầu một cách thích hợp.

### Nhiệm vụ:
{}

### Đầu vào:
{}

### Câu trả lời:
{}"""

# Hàm tạo phản hồi từ mô hình
async def generate_response(user_input):
    try:
        inputs = tokenizer(
            [
                alpaca_prompt.format(
                    "If you are a doctor, please answer the medical questions based on the patient's description.",
                    user_input,
                    ""
                )
            ],
            return_tensors="pt"
        ).to(device)

        text_streamer = TextStreamer(tokenizer)
        outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128*2)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Chỉ lấy phần sau "Câu trả lời:"
        answer_start = response.find("Câu trả lời:")
        if answer_start != -1:
            return response[answer_start + len("Câu trả lời:"):].strip()
        else:
            return response.strip()  # Nếu không tìm thấy, trả về toàn bộ phản hồi

    except Exception as e:
        print(f"Lỗi xảy ra: {e}")  # In lỗi ra để dễ debug nếu cần
        return "Đã có lỗi xảy ra, vui lòng thử lại sau."


# Hàm xử lý khi người dùng gửi lệnh /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Xin chào! Tôi là chatbot hỗ trợ tư vấn y tế. Hãy gửi câu hỏi của bạn.")

# Hàm xử lý tin nhắn từ người dùng
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_input = update.message.text
    response = await generate_response(user_input)
    await update.message.reply_text(response)

# Hàm chính để khởi động bot
def main():
    TOKEN = '7556701680:AAEK4XVtB7lvHOFMpmPKGJndyE8UVoI-kjA'            # API lấy từ BotFather
    app = Application.builder().token(TOKEN).build()

    # Thêm handler cho lệnh /start
    app.add_handler(CommandHandler("start", start))

    # Thêm handler cho tin nhắn văn bản
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Khởi động bot
    app.run_polling()

if __name__ == '__main__':
    main()


==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Dưới đây là hướng dẫn mô tả một nhiệm vụ. Viết một phản hồi hoàn thành yêu cầu một cách thích hợp.

### Nhiệm vụ:
If you are a doctor, please answer the medical questions based on the patient's description.

### Đầu vào:
I have a headache

### Câu trả lời:
Hi, Thanks for your query. I have gone through your query. I understand your concern. Headache is a common symptom. It can be due to many reasons. It can be due to stress, anxiety, tension, depression, hormonal imbalance, sinusitis, migraine, tension headache, etc. It can be due